# Heat System

District heating with thermal storage and time-varying prices.

This notebook introduces:

- **Storage**: Thermal buffer tanks with charging/discharging
- **Time series data**: Using real demand profiles
- **Multiple components**: Combining boiler, storage, and loads
- **Result visualization**: Heatmaps, balance plots, and charge states

## Setup

In [1]:
import pandas as pd
import xarray as xr

import flixopt as fx

fx.CONFIG.notebook()

flixopt.config.CONFIG

## Define Time Horizon and Demand

We model one week with hourly resolution. The office has typical weekday patterns:

In [2]:
from data.tutorial_data import get_heat_system_data

data = get_heat_system_data()
timesteps = data['timesteps']
heat_demand = data['heat_demand']
gas_price = data['gas_price']

In [3]:
# Visualize the demand pattern with plotly
demand_ds = xr.Dataset(
    {
        'Heat Demand': xr.DataArray(heat_demand, dims=['time'], coords={'time': timesteps}),
    }
)
demand_ds.plotly.line(x='time', title='Office Heat Demand Profile')

## Define Gas Prices

Gas prices vary with time-of-use tariffs:

In [4]:
# Visualize time-of-use gas prices with plotly
price_ds = xr.Dataset(
    {
        'Gas Price': xr.DataArray(gas_price, dims=['time'], coords={'time': timesteps}),
    }
)
price_ds.plotly.line(x='time', title='Gas Price [€/kWh]')

## Build the Energy System

The system includes:
- Gas boiler (150 kW thermal capacity)
- Thermal storage tank (500 kWh capacity)
- Office building heat demand

```
Gas Grid ──► [Gas] ──► Boiler ──► [Heat] ◄──► Storage
                                    │
                                    ▼
                                 Office
```

In [5]:
flow_system = fx.FlowSystem(timesteps)
flow_system.add_carriers(
    fx.Carrier('gas', '#3498db', 'kW'),
    fx.Carrier('heat', '#e74c3c', 'kW'),
)
flow_system.add_elements(
    # === Buses ===
    fx.Bus('Gas', carrier='gas'),
    fx.Bus('Heat', carrier='heat'),
    # === Effect ===
    fx.Effect('costs', '€', 'Operating Costs', is_standard=True, is_objective=True),
    # === Gas Supply with time-varying price ===
    fx.Source(
        'GasGrid',
        outputs=[fx.Flow('Gas', bus='Gas', size=500, effects_per_flow_hour=gas_price)],
    ),
    # === Gas Boiler: 150 kW, 92% efficiency ===
    fx.linear_converters.Boiler(
        'Boiler',
        thermal_efficiency=0.92,
        thermal_flow=fx.Flow('Heat', bus='Heat', size=150),
        fuel_flow=fx.Flow('Gas', bus='Gas'),
    ),
    # === Thermal Storage: 500 kWh tank ===
    fx.Storage(
        'ThermalStorage',
        capacity_in_flow_hours=500,  # 500 kWh capacity
        initial_charge_state=250,  # Start half-full
        minimal_final_charge_state=200,  # End with at least 200 kWh
        eta_charge=0.98,  # 98% charging efficiency
        eta_discharge=0.98,  # 98% discharging efficiency
        relative_loss_per_hour=0.005,  # 0.5% heat loss per hour
        charging=fx.Flow('Charge', bus='Heat', size=100),  # Max 100 kW charging
        discharging=fx.Flow('Discharge', bus='Heat', size=100),  # Max 100 kW discharging
    ),
    # === Office Heat Demand ===
    fx.Sink(
        'Office',
        inputs=[fx.Flow('Heat', bus='Heat', size=1, fixed_relative_profile=heat_demand)],
    ),
)

## Run Optimization

In [6]:
flow_system.optimize(fx.solvers.HighsSolver(mip_gap=0.01));

## Analyze Results

### Heat Balance

See how the boiler and storage work together to meet demand:

In [7]:
flow_system.statistics.plot.balance('Heat')

PlotResult(data=<xarray.Dataset> Size: 7kB
Dimensions:                    (time: 169)
Coordinates:
  * time                       (time) datetime64[ns] 1kB 2024-01-15 ... 2024-...
Data variables:
    Boiler(Heat)               (time) float64 1kB -32.48 -29.31 ... -124.5 nan
    ThermalStorage(Discharge)  (time) float64 1kB -0.0 1.758e-13 ... -0.0 nan
    ThermalStorage(Charge)     (time) float64 1kB 0.0 4.798e-14 ... 100.0 nan
    Office(Heat)               (time) float64 1kB 32.48 29.31 ... 24.48 nan, figure=Figure({
    'data': [{'fillcolor': '#EF553B',
              'fillpattern': {'shape': ''},
              'hovertemplate': '<b>%{fullData.name}</b>: %{y:.1f} kW<extra></extra>',
              'legendgroup': 'Boiler(Heat)',
              'line': {'color': '#EF553B', 'shape': 'hv', 'width': 0},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Boiler(Heat)',
              'orientation': 'v',
              'showlegend': True,
              'stackgroup': 'negative',
              'type': 'scatter',
              'x': array(['2024-01-15T00:00:00.000000000', '2024-01-15T01:00:00.000000000',
                          '2024-01-15T02:00:00.000000000', '2024-01-15T03:00:00.000000000',
                          '2024-01-15T04:00:00.000000000', '2024-01-15T05:00:00.000000000',
                          '2024-01-15T06:00:00.000000000', '2024-01-15T07:00:00.000000000',
                          '2024-01-15T08:00:00.000000000', '2024-01-15T09:00:00.000000000',
                          '2024-01-15T10:00:00.000000000', '2024-01-15T11:00:00.000000000',
                          '2024-01-15T12:00:00.000000000', '2024-01-15T13:00:00.000000000',
                          '2024-01-15T14:00:00.000000000', '2024-01-15T15:00:00.000000000',
                          '2024-01-15T16:00:00.000000000', '2024-01-15T17:00:00.000000000',
                          '2024-01-15T18:00:00.000000000', '2024-01-15T19:00:00.000000000',
                          '2024-01-15T20:00:00.000000000', '2024-01-15T21:00:00.000000000',
                          '2024-01-15T22:00:00.000000000', '2024-01-15T23:00:00.000000000',
                          '2024-01-16T00:00:00.000000000', '2024-01-16T01:00:00.000000000',
                          '2024-01-16T02:00:00.000000000', '2024-01-16T03:00:00.000000000',
                          '2024-01-16T04:00:00.000000000', '2024-01-16T05:00:00.000000000',
                          '2024-01-16T06:00:00.000000000', '2024-01-16T07:00:00.000000000',
                          '2024-01-16T08:00:00.000000000', '2024-01-16T09:00:00.000000000',
                          '2024-01-16T10:00:00.000000000', '2024-01-16T11:00:00.000000000',
                          '2024-01-16T12:00:00.000000000', '2024-01-16T13:00:00.000000000',
                          '2024-01-16T14:00:00.000000000', '2024-01-16T15:00:00.000000000',
                          '2024-01-16T16:00:00.000000000', '2024-01-16T17:00:00.000000000',
                          '2024-01-16T18:00:00.000000000', '2024-01-16T19:00:00.000000000',
                          '2024-01-16T20:00:00.000000000', '2024-01-16T21:00:00.000000000',
                          '2024-01-16T22:00:00.000000000', '2024-01-16T23:00:00.000000000',
                          '2024-01-17T00:00:00.000000000', '2024-01-17T01:00:00.000000000',
                          '2024-01-17T02:00:00.000000000', '2024-01-17T03:00:00.000000000',
                          '2024-01-17T04:00:00.000000000', '2024-01-17T05:00:00.000000000',
                          '2024-01-17T06:00:00.000000000', '2024-01-17T07:00:00.000000000',
                          '2024-01-17T08:00:00.000000000', '2024-01-17T09:00:00.000000000',
                          '2024-01-17T10:00:00.000000000', '2024-01-17T11:00:00.000000000',
                          '2024-01-17T12:00:00.000000000', '2024-01-17T13:00:00.000000000',
                          '2024-01-17T14:00:00.000000000', '2024-01-17T15:00:00.0000

### Storage Charge State

Track how the storage level varies over time:

In [8]:
flow_system.statistics.plot.balance('ThermalStorage')

PlotResult(data=<xarray.Dataset> Size: 4kB
Dimensions:                    (time: 169)
Coordinates:
  * time                       (time) datetime64[ns] 1kB 2024-01-15 ... 2024-...
Data variables:
    ThermalStorage(Charge)     (time) float64 1kB -0.0 -4.798e-14 ... -100.0 nan
    ThermalStorage(Discharge)  (time) float64 1kB 0.0 -1.758e-13 ... 0.0 nan, figure=Figure({
    'data': [{'fillcolor': '#e74c3c',
              'fillpattern': {'shape': ''},
              'hovertemplate': '<b>%{fullData.name}</b>: %{y:.1f} kW<extra></extra>',
              'legendgroup': 'ThermalStorage(Charge)',
              'line': {'color': '#e74c3c', 'shape': 'hv', 'width': 0},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'ThermalStorage(Charge)',
              'orientation': 'v',
              'showlegend': True,
              'stackgroup': 'negative',
              'type': 'scatter',
              'x': array(['2024-01-15T00:00:00.000000000', '2024-01-15T01:00:00.000000000',
                          '2024-01-15T02:00:00.000000000', '2024-01-15T03:00:00.000000000',
                          '2024-01-15T04:00:00.000000000', '2024-01-15T05:00:00.000000000',
                          '2024-01-15T06:00:00.000000000', '2024-01-15T07:00:00.000000000',
                          '2024-01-15T08:00:00.000000000', '2024-01-15T09:00:00.000000000',
                          '2024-01-15T10:00:00.000000000', '2024-01-15T11:00:00.000000000',
                          '2024-01-15T12:00:00.000000000', '2024-01-15T13:00:00.000000000',
                          '2024-01-15T14:00:00.000000000', '2024-01-15T15:00:00.000000000',
                          '2024-01-15T16:00:00.000000000', '2024-01-15T17:00:00.000000000',
                          '2024-01-15T18:00:00.000000000', '2024-01-15T19:00:00.000000000',
                          '2024-01-15T20:00:00.000000000', '2024-01-15T21:00:00.000000000',
                          '2024-01-15T22:00:00.000000000', '2024-01-15T23:00:00.000000000',
                          '2024-01-16T00:00:00.000000000', '2024-01-16T01:00:00.000000000',
                          '2024-01-16T02:00:00.000000000', '2024-01-16T03:00:00.000000000',
                          '2024-01-16T04:00:00.000000000', '2024-01-16T05:00:00.000000000',
                          '2024-01-16T06:00:00.000000000', '2024-01-16T07:00:00.000000000',
                          '2024-01-16T08:00:00.000000000', '2024-01-16T09:00:00.000000000',
                          '2024-01-16T10:00:00.000000000', '2024-01-16T11:00:00.000000000',
                          '2024-01-16T12:00:00.000000000', '2024-01-16T13:00:00.000000000',
                          '2024-01-16T14:00:00.000000000', '2024-01-16T15:00:00.000000000',
                          '2024-01-16T16:00:00.000000000', '2024-01-16T17:00:00.000000000',
                          '2024-01-16T18:00:00.000000000', '2024-01-16T19:00:00.000000000',
                          '2024-01-16T20:00:00.000000000', '2024-01-16T21:00:00.000000000',
                          '2024-01-16T22:00:00.000000000', '2024-01-16T23:00:00.000000000',
                          '2024-01-17T00:00:00.000000000', '2024-01-17T01:00:00.000000000',
                          '2024-01-17T02:00:00.000000000', '2024-01-17T03:00:00.000000000',
                          '2024-01-17T04:00:00.000000000', '2024-01-17T05:00:00.000000000',
                          '2024-01-17T06:00:00.000000000', '2024-01-17T07:00:00.000000000',
                          '2024-01-17T08:00:00.000000000', '2024-01-17T09:00:00.000000000',
                          '2024-01-17T10:00:00.000000000', '2024-01-17T11:00:00.000000000',
                          '2024-01-17T12:00:00.000000000', '2024-01-17T13:00:00.000000000',
                          '2024-01-17T14:00:00.000000000', '2024-01-17T15:00:00.000000000',
                          '2024-01-17T16:00:00.000000000', '2024-01-17T17:00:00.000000000',
                          '2024-0

### Heatmap Visualization

Heatmaps show patterns across hours and days:

In [9]:
flow_system.statistics.plot.heatmap('Boiler(Heat)')

PlotResult(data=<xarray.Dataset> Size: 2kB
Dimensions:    (timeframe: 8, timestep: 24)
Coordinates:
  * timeframe  (timeframe) object 64B '2024-01-15' '2024-01-16' ... '2024-01-22'
  * timestep   (timestep) object 192B '00:00' '01:00' ... '22:00' '23:00'
Data variables:
    value      (timestep, timeframe) float64 2kB 32.48 42.84 47.28 ... 124.5 nan, figure=Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'timeframe: %{x}<br>timestep: %{y}<br>Boiler(Heat)|flow_rate: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'x': array(['2024-01-15', '2024-01-16', '2024-01-17', '2024-01-18', '2024-01-19',
                          '2024-01-20', '2024-01-21', '2024-01-22'], dtype=object),
              'xaxis': 'x',
              'y': array(['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00',
                          '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00',
                          '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00'],
                         dtype=object),
              'yaxis': 'y',
              'z': {'bdata': ('5ZuWpeU9QECTnmEA1mtFQBN9bxYopE' ... '//////M0DuufhH+R5fQAAAAAAAAPh/'),
                    'dtype': 'f8',
                    'shape': '24, 8'}}],
    'layout': {'coloraxis': {'cmax': 137.82321827907003,
                             'cmin': 0.0,
                             'colorbar': {'title': {'text': 'Boiler(Heat)|flow_rate'}},
                             'colorscale': [[0.0, '#30123b'],
                                            [0.07142857142857142, '#4145ab'],
                                            [0.14285714285714285, '#4675ed'],
                                            [0.21428571428571427, '#39a2fc'],
                                            [0.2857142857142857, '#1bcfd4'],
                                            [0.35714285714285715, '#24eca6'],
                                            [0.42857142857142855, '#61fc6c'], [0.5,
                                            '#a4fc3b'], [0.5714285714285714,
                                            '#d1e834'], [0.6428571428571429,
                                            '#f3c63a'], [0.7142857142857143,
                                            '#fe9b2d'], [0.7857142857142857,
                                            '#f36315'], [0.8571428571428571,
                                            '#d93806'], [0.9285714285714286,
                                            '#b11901'], [1.0, '#7a0402']]},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'timeframe'}},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'domain': [0.0, 1.0], 'title': {'text': 'timestep'}}}
}))

In [10]:
flow_system.statistics.plot.heatmap('ThermalStorage')

PlotResult(data=<xarray.Dataset> Size: 2kB
Dimensions:    (timeframe: 8, timestep: 24)
Coordinates:
  * timeframe  (timeframe) object 64B '2024-01-15' '2024-01-16' ... '2024-01-22'
  * timestep   (timestep) object 192B '00:00' '01:00' ... '22:00' '23:00'
Data variables:
    value      (timestep, timeframe) float64 2kB 250.0 1.378e-14 ... 102.5 nan, figure=Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': ('timeframe: %{x}<br>timestep: %' ... 'rge_state: %{z}<extra></extra>'),
              'name': '0',
              'type': 'heatmap',
              'x': array(['2024-01-15', '2024-01-16', '2024-01-17', '2024-01-18', '2024-01-19',
                          '2024-01-20', '2024-01-21', '2024-01-22'], dtype=object),
              'xaxis': 'x',
              'y': array(['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00',
                          '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00',
                          '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00'],
                         dtype=object),
              'yaxis': 'y',
              'z': {'bdata': ('AAAAAABAb0BQ0yVzkQgPPXbSCclOzz' ... 'AAAAAAAAD65ELUzaBZQAAAAAAAAPh/'),
                    'dtype': 'f8',
                    'shape': '24, 8'}}],
    'layout': {'coloraxis': {'cmax': 500.0,
                             'cmin': -9.647140229519998e-14,
                             'colorbar': {'title': {'text': 'ThermalStorage|charge_state'}},
                             'colorscale': [[0.0, '#30123b'],
                                            [0.07142857142857142, '#4145ab'],
                                            [0.14285714285714285, '#4675ed'],
                                            [0.21428571428571427, '#39a2fc'],
                                            [0.2857142857142857, '#1bcfd4'],
                                            [0.35714285714285715, '#24eca6'],
                                            [0.42857142857142855, '#61fc6c'], [0.5,
                                            '#a4fc3b'], [0.5714285714285714,
                                            '#d1e834'], [0.6428571428571429,
                                            '#f3c63a'], [0.7142857142857143,
                                            '#fe9b2d'], [0.7857142857142857,
                                            '#f36315'], [0.8571428571428571,
                                            '#d93806'], [0.9285714285714286,
                                            '#b11901'], [1.0, '#7a0402']]},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'timeframe'}},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'domain': [0.0, 1.0], 'title': {'text': 'timestep'}}}
}))

### Cost Analysis

In [11]:
total_heat = heat_demand.sum()

pd.DataFrame(
    {
        'Total operating costs [EUR]': flow_system.solution['costs'].item(),
        'Total heat delivered [kWh]': total_heat,
        'Average cost [ct/kWh]': flow_system.solution['costs'].item() / total_heat * 100,
    },
    index=['Value'],
).T

,Value
Total operating costs [EUR],558.830517
Total heat delivered [kWh],8007.338386
Average cost [ct/kWh],6.978980


### Flow Rates and Charge States

Visualize all flow rates and storage charge states:

In [12]:
# Plot all flow rates
flow_system.statistics.plot.flows()

PlotResult(data=<xarray.Dataset> Size: 9kB
Dimensions:                    (time: 169)
Coordinates:
  * time                       (time) datetime64[ns] 1kB 2024-01-15 ... 2024-...
Data variables:
    GasGrid(Gas)               (time) float64 1kB 35.31 31.86 ... 135.3 nan
    Boiler(Gas)                (time) float64 1kB 35.31 31.86 ... 135.3 nan
    Boiler(Heat)               (time) float64 1kB 32.48 29.31 ... 124.5 nan
    ThermalStorage(Charge)     (time) float64 1kB 0.0 4.798e-14 ... 100.0 nan
    ThermalStorage(Discharge)  (time) float64 1kB 0.0 -1.758e-13 ... 0.0 nan
    Office(Heat)               (time) float64 1kB 32.48 29.31 ... 24.48 nan, figure=Figure({
    'data': [{'hovertemplate': 'variable=GasGrid(Gas)<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'GasGrid(Gas)',
              'line': {'color': '#636EFA', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'GasGrid(Gas)',
              'showlegend': True,
              'type': 'scattergl',
              'x': array(['2024-01-15T00:00:00.000000000', '2024-01-15T01:00:00.000000000',
                          '2024-01-15T02:00:00.000000000', '2024-01-15T03:00:00.000000000',
                          '2024-01-15T04:00:00.000000000', '2024-01-15T05:00:00.000000000',
                          '2024-01-15T06:00:00.000000000', '2024-01-15T07:00:00.000000000',
                          '2024-01-15T08:00:00.000000000', '2024-01-15T09:00:00.000000000',
                          '2024-01-15T10:00:00.000000000', '2024-01-15T11:00:00.000000000',
                          '2024-01-15T12:00:00.000000000', '2024-01-15T13:00:00.000000000',
                          '2024-01-15T14:00:00.000000000', '2024-01-15T15:00:00.000000000',
                          '2024-01-15T16:00:00.000000000', '2024-01-15T17:00:00.000000000',
                          '2024-01-15T18:00:00.000000000', '2024-01-15T19:00:00.000000000',
                          '2024-01-15T20:00:00.000000000', '2024-01-15T21:00:00.000000000',
                          '2024-01-15T22:00:00.000000000', '2024-01-15T23:00:00.000000000',
                          '2024-01-16T00:00:00.000000000', '2024-01-16T01:00:00.000000000',
                          '2024-01-16T02:00:00.000000000', '2024-01-16T03:00:00.000000000',
                          '2024-01-16T04:00:00.000000000', '2024-01-16T05:00:00.000000000',
                          '2024-01-16T06:00:00.000000000', '2024-01-16T07:00:00.000000000',
                          '2024-01-16T08:00:00.000000000', '2024-01-16T09:00:00.000000000',
                          '2024-01-16T10:00:00.000000000', '2024-01-16T11:00:00.000000000',
                          '2024-01-16T12:00:00.000000000', '2024-01-16T13:00:00.000000000',
                          '2024-01-16T14:00:00.000000000', '2024-01-16T15:00:00.000000000',
                          '2024-01-16T16:00:00.000000000', '2024-01-16T17:00:00.000000000',
                          '2024-01-16T18:00:00.000000000', '2024-01-16T19:00:00.000000000',
                          '2024-01-16T20:00:00.000000000', '2024-01-16T21:00:00.000000000',
                          '2024-01-16T22:00:00.000000000', '2024-01-16T23:00:00.000000000',
                          '2024-01-17T00:00:00.000000000', '2024-01-17T01:00:00.000000000',
                          '2024-01-17T02:00:00.000000000', '2024-01-17T03:00:00.000000000',
                          '2024-01-17T04:00:00.000000000', '2024-01-17T05:00:00.000000000',
                          '2024-01-17T06:00:00.000000000', '2024-01-17T07:00:00.000000000',
                          '2024-01-17T08:00:00.000000000', '2024-01-17T09:00:00.000000000',
                          '2024-01-17T10:00:00.000000000', '2024-01-17T11:00:00.000000000',
                          '2024-01-17T12:00:00.000000000', '2024-01-17T13:00:00.000000000',
                          '2024-01-17T14:00:00.000000000', '2024-01-17T15:00:00.00000000

In [13]:
# Plot storage charge states
flow_system.statistics.plot.storage('ThermalStorage')

PlotResult(data=<xarray.Dataset> Size: 5kB
Dimensions:                    (time: 169)
Coordinates:
  * time                       (time) datetime64[ns] 1kB 2024-01-15 ... 2024-...
Data variables:
    ThermalStorage(Charge)     (time) float64 1kB 0.0 4.798e-14 ... 100.0 nan
    ThermalStorage(Discharge)  (time) float64 1kB -0.0 1.758e-13 ... -0.0 nan
    charge_state               (time) float64 1kB 250.0 248.8 ... 102.5 200.0, figure=Figure({
    'data': [{'fillcolor': '#e74c3c',
              'fillpattern': {'shape': ''},
              'hovertemplate': '<b>%{fullData.name}</b>: %{y:.1f} kW<extra></extra>',
              'legendgroup': 'ThermalStorage(Charge)',
              'line': {'color': '#e74c3c', 'shape': 'hv', 'width': 0},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'ThermalStorage(Charge)',
              'orientation': 'v',
              'showlegend': True,
              'stackgroup': 'positive',
              'type': 'scatter',
              'x': array(['2024-01-15T00:00:00.000000000', '2024-01-15T01:00:00.000000000',
                          '2024-01-15T02:00:00.000000000', '2024-01-15T03:00:00.000000000',
                          '2024-01-15T04:00:00.000000000', '2024-01-15T05:00:00.000000000',
                          '2024-01-15T06:00:00.000000000', '2024-01-15T07:00:00.000000000',
                          '2024-01-15T08:00:00.000000000', '2024-01-15T09:00:00.000000000',
                          '2024-01-15T10:00:00.000000000', '2024-01-15T11:00:00.000000000',
                          '2024-01-15T12:00:00.000000000', '2024-01-15T13:00:00.000000000',
                          '2024-01-15T14:00:00.000000000', '2024-01-15T15:00:00.000000000',
                          '2024-01-15T16:00:00.000000000', '2024-01-15T17:00:00.000000000',
                          '2024-01-15T18:00:00.000000000', '2024-01-15T19:00:00.000000000',
                          '2024-01-15T20:00:00.000000000', '2024-01-15T21:00:00.000000000',
                          '2024-01-15T22:00:00.000000000', '2024-01-15T23:00:00.000000000',
                          '2024-01-16T00:00:00.000000000', '2024-01-16T01:00:00.000000000',
                          '2024-01-16T02:00:00.000000000', '2024-01-16T03:00:00.000000000',
                          '2024-01-16T04:00:00.000000000', '2024-01-16T05:00:00.000000000',
                          '2024-01-16T06:00:00.000000000', '2024-01-16T07:00:00.000000000',
                          '2024-01-16T08:00:00.000000000', '2024-01-16T09:00:00.000000000',
                          '2024-01-16T10:00:00.000000000', '2024-01-16T11:00:00.000000000',
                          '2024-01-16T12:00:00.000000000', '2024-01-16T13:00:00.000000000',
                          '2024-01-16T14:00:00.000000000', '2024-01-16T15:00:00.000000000',
                          '2024-01-16T16:00:00.000000000', '2024-01-16T17:00:00.000000000',
                          '2024-01-16T18:00:00.000000000', '2024-01-16T19:00:00.000000000',
                          '2024-01-16T20:00:00.000000000', '2024-01-16T21:00:00.000000000',
                          '2024-01-16T22:00:00.000000000', '2024-01-16T23:00:00.000000000',
                          '2024-01-17T00:00:00.000000000', '2024-01-17T01:00:00.000000000',
                          '2024-01-17T02:00:00.000000000', '2024-01-17T03:00:00.000000000',
                          '2024-01-17T04:00:00.000000000', '2024-01-17T05:00:00.000000000',
                          '2024-01-17T06:00:00.000000000', '2024-01-17T07:00:00.000000000',
                          '2024-01-17T08:00:00.000000000', '2024-01-17T09:00:00.000000000',
                          '2024-01-17T10:00:00.000000000', '2024-01-17T11:00:00.000000000',
                          '2024-01-17T12:00:00.000000000', '2024-01-17T13:00:00.000000000',
                          '2024-01-17T14:00:00.000000000', '2024-01-17T15:00:00.000000000',
                          '2024-01-17T16:00:00.00

### Energy Flow Sankey

A Sankey diagram visualizes the total energy flows through the system:

In [14]:
flow_system.statistics.plot.sankey.flows()

PlotResult(data=<xarray.Dataset> Size: 1kB
Dimensions:  (link: 6)
Coordinates:
  * link     (link) int64 48B 0 1 2 3 4 5
    source   (link) <U14 336B 'Gas' 'Boiler' ... 'Heat' 'ThermalStorage'
    target   (link) <U14 336B 'Boiler' 'Heat' 'Gas' ... 'ThermalStorage' 'Heat'
    label    (link) <U25 600B 'Boiler(Gas)' ... 'ThermalStorage(Discharge)'
    carrier  (link) <U4 96B 'gas' 'heat' 'gas' 'heat' 'heat' 'heat'
Data variables:
    value    (link) float64 48B 8.937e+03 8.222e+03 ... 3.457e+03 3.243e+03, figure=Figure({
    'data': [{'link': {'color': [rgba(52, 152, 219, 0.4), rgba(231, 76, 60, 0.4),
                                 rgba(52, 152, 219, 0.4), rgba(231, 76, 60, 0.4),
                                 rgba(231, 76, 60, 0.4), rgba(231, 76, 60, 0.4)],
                       'label': [Boiler(Gas), Boiler(Heat), GasGrid(Gas),
                                 Office(Heat), ThermalStorage(Charge),
                                 ThermalStorage(Discharge)],
                       'source': [2, 4, 3, 1, 1, 5],
                       'target': [4, 1, 2, 0, 5, 1],
                       'value': [8936.66540578717, 8221.732173324197,
                                 8936.66540578717, 8007.3383864452235,
                                 3457.1827352482123, 3242.7889483692406]},
              'node': {'color': [#AB63FA, #e74c3c, #3498db, #636EFA, #EF553B,
                                 #00CC96],
                       'label': [Office, Heat, Gas, GasGrid, Boiler,
                                 ThermalStorage],
                       'line': {'color': 'black', 'width': 0.5},
                       'pad': 15,
                       'thickness': 20},
              'type': 'sankey'}],
    'layout': {'template': '...', 'title': {'text': 'Energy Flow'}}
}))

## Key Insights

The optimization reveals how storage enables **load shifting**:

1. **Charge during off-peak**: When gas is cheap (night), the boiler runs at higher output to charge the storage
2. **Discharge during peak**: During expensive periods, storage supplements the boiler
3. **Weekend patterns**: Lower demand allows more storage cycling

## Summary

You learned how to:

- Add **Storage** components with efficiency and losses
- Use **time-varying prices** in effects
- Visualize results with **heatmaps** and **balance plots**
- Access raw data via **statistics.flow_rates** and **statistics.charge_states**

### Next Steps

- **[03-investment-optimization](03-investment-optimization.ipynb)**: Optimize storage size
- **[04-operational-constraints](04-operational-constraints.ipynb)**: Add startup costs and minimum run times